In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [78]:
url = 'https://merolagani.com/CompanyDetail.aspx?symbol=GBIME#0'

In [79]:
response = requests.get(url)
response

<Response [200]>

In [80]:
response = response.content

In [81]:
soup = BeautifulSoup(response, 'html.parser')

In [82]:
soup


<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml">
<head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><title>
	merolagani - Global IME Bank Limited (GBIME) Detail, Announcements, News, Price History, Floorsheet
</title><link href="/bundles/css/main200?v=XBmv3Nbj66DV6lzAxQCBgaIOrn903Diuut8lsSb3n9k1" rel="stylesheet"/>
<link href="/bundles/css/site205?v=JDFYx0XBsjx2PJsB70xsp6n6BXVCmnegzsdX8OWN36Y1" rel="stylesheet"/>
<link href="/Content/Services.css" rel="stylesheet"/>
<link href="/Content/owl.carousel.min.css" rel="stylesheet"/>
<link href="/Content/lightslider/css/lightslider.css" rel="stylesheet"/>
<script src="/scripts/jquery/jquery-1.11.0.min.js"></script>
<script src="/scripts/jquery/jquery-ui.js"></script>
<script src="/Content/lightslider/js/lightslider.js"></script>
<!--[if lt IE 9]>
      <script src="https://oss.maxcdn.com/html5shiv/3.7.2/html5shiv.min.js"></script>

In [83]:
table= soup.find_all('table', class_='table table-bordered table-striped table-hover')

In [84]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

url = "https://merolagani.com/CompanyDetail.aspx?symbol=GBIME#0"
driver = webdriver.Chrome() # create a Chrome driver
driver.get(url) # open the URL in the browser
wait = WebDriverWait(driver, 10) # wait for up to 10 seconds
price_history = wait.until(EC.element_to_be_clickable((By.ID, "ctl00_ContentPlaceHolder1_CompanyDetail1_lnkHistoryTab")))
 # find the price history div element
price_history.click() # click on the price history div element
html_content = driver.page_source # get the HTML source code
driver.quit() # close the browser
soup = BeautifulSoup(html_content, "html.parser") # parse the HTML with BeautifulSoup
table = soup.find_all("table", class_='table table-bordered table-striped table-hover') # find the table element with the price history data


print(table) # print the table element


[<table class="table table-bordered table-striped table-hover">
</table>, <table class="table table-bordered table-striped table-hover">
</table>, <table class="table table-bordered table-striped table-hover">
</table>, <table class="table table-bordered table-striped table-hover">
</table>, <table class="table table-bordered table-striped table-hover">
</table>, <table class="table table-bordered table-striped table-hover">
</table>, <table class="table table-bordered table-striped table-hover">
</table>, <table class="table table-bordered table-striped table-hover">
</table>, <table class="table table-bordered table-striped table-hover">
</table>]


In [97]:
new_table = soup.find ("table", {"class": "table-responsive"})

In [103]:
spans = new_table.find_all("span") 

AttributeError: 'NoneType' object has no attribute 'find_all'

In [93]:
soup.find_all('span')

[<span class="float-right">We'd like to send you notifications for the latest news and updates.</span>,
 <span class="glyphicon glyphicon-one-fine-white-dot">ENGLISH</span>,
 <span class="icon-angle-down"></span>,
 <span class="sr-only">Toggle navigation</span>,
 <span class="icon-bar"></span>,
 <span class="icon-bar"></span>,
 <span class="icon-bar"></span>,
 <span class="pull-right verify-position" id="username-verify"></span>,
 <span class="icon-search"></span>,
 <span class="icon-home icon-2x" style="line-height: 20px"></span>,
 <span class="badge-info"><span class="glyphicon glyphicon-flag blink" style="color: #edaa4e"></span></span>,
 <span class="glyphicon glyphicon-flag blink" style="color: #edaa4e"></span>,
 <span class="badge-info"><span class="glyphicon glyphicon-flag blink" style="color: #edaa4e"></span></span>,
 <span class="glyphicon glyphicon-flag blink" style="color: #edaa4e"></span>,
 <span class="badge-info"><span class="glyphicon glyphicon-flag blink" style="color: #